In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Transaction-level Dataset


In [2]:
## Load the data into a DataFrame
transactions=pd.read_csv('/Users/austinlasseter/DSI-EC-2/projects/datasets/iowa_liquor_short.csv')

In [3]:
# Drop the columns I don't plan to use
transactions=transactions.drop(['Area_name', 'State', 'Unnamed: 0', 'Date', 'County Number'], axis=1);

In [4]:
# Some of these names are really long.
transactions.columns

Index(['FIPS', 'Rural_urban_continuum_code_2013', 'Unemployment_rate_2016',
       'Median_Household_Income_2016', 'CENSUS_2010_POP', 'County',
       'Store Number', 'State Bottle Cost', 'State Bottle Retail',
       'Bottles Sold', 'Year'],
      dtype='object')

In [5]:
# Shorten those names, and make them lower-case (this is best practice)
new_cols=['fips', 'metro', 'unemployment',
       'income', 'population', 'county',
       'store', 'cost', 'retail',
       'bottles', 'year']
transactions.columns=new_cols
transactions.head()

,fips,metro,unemployment,income,population,county,store,cost,retail,bottles,year
0,19001,8.0,3.0,53416.0,7682,Adair,3461,$5.23,$7.85,12,2015
1,19001,8.0,3.0,53416.0,7682,Adair,3461,$6.74,$10.11,4,2015
2,19001,8.0,3.0,53416.0,7682,Adair,3461,$4.00,$6.00,3,2015
3,19001,8.0,3.0,53416.0,7682,Adair,4620,$5.48,$8.22,24,2015
4,19001,8.0,3.0,53416.0,7682,Adair,4654,$12.05,$18.08,12,2015


In [6]:
# Convert cost and retail into numeric values
transactions['cost']=transactions['cost'].replace( '[\$,)]','', regex=True ).astype(float)
transactions['retail']=transactions['retail'].replace( '[\$,)]','', regex=True ).astype(float)

In [7]:
# Make sure that "bottles" is an integer
transactions.bottles.dtype

dtype('int64')

In [8]:
# restrict the dataset only to 2015 sales (so we have a complete year of information)
transactions=transactions[transactions['year']==2015]

In [9]:
# Calculate transaction-level profit
transactions['profit_per_sale'] = (transactions['retail'] - transactions['cost'])*transactions['bottles']
transactions.head()

,fips,metro,unemployment,income,population,county,store,cost,retail,bottles,year,profit_per_sale
0,19001,8.0,3.0,53416.0,7682,Adair,3461,5.23,7.85,12,2015,31.44
1,19001,8.0,3.0,53416.0,7682,Adair,3461,6.74,10.11,4,2015,13.48
2,19001,8.0,3.0,53416.0,7682,Adair,3461,4.00,6.00,3,2015,6.00
3,19001,8.0,3.0,53416.0,7682,Adair,4620,5.48,8.22,24,2015,65.76
4,19001,8.0,3.0,53416.0,7682,Adair,4654,12.05,18.08,12,2015,72.36


In [10]:
# Calculate store-level annual profit
x=transactions['profit_per_sale'].groupby(transactions['store']).sum()
print(len(x))
profit=pd.DataFrame(x);

1375


In [11]:
# Reset the index so it isn't "store"
profit.reset_index(level=0, inplace=True)
profit.head()

,store,profit_per_sale
0,2106,478695.22
1,2113,28693.18
2,2130,369675.74
3,2152,24143.45
4,2178,93235.51


# Store-level data

In [12]:
# How many transactions are there the entire state? There are at least 3 ways to do this.
print(len(transactions.groupby('store')))
print(transactions.store.nunique())
print(len(transactions.store.unique()))

1375
1375
1375


In [13]:
# create a new dataset where every row is a store
stores=transactions.drop(['cost', 'year', 'profit_per_sale'], axis=1).groupby('store').mean()
print(len(stores))
stores.head()

1375


,fips,metro,unemployment,income,population,retail,bottles
store,,,,,,,
2106,19013.0,3.0,4.7,50887.0,131090.0,16.225173,19.588247
2113,19187.0,5.0,4.0,48013.0,38013.0,15.963071,4.763409
2130,19013.0,3.0,4.7,50887.0,131090.0,15.423895,18.374778
2152,19033.0,5.0,3.7,53109.0,44151.0,12.897199,4.131010
2178,19005.0,6.0,5.0,49439.0,14330.0,15.046966,7.699587


In [14]:
# Reset the index so it isn't "store"
stores.reset_index(level=0, inplace=True)

In [15]:
# Merge back in the "annual profit" variable
stores=pd.merge(stores, profit, on='store')

In [16]:
# Rename some of the columns
print(stores.columns)
new_cols=['fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store']
stores.columns=new_cols

Index(['store', 'fips', 'metro', 'unemployment', 'income', 'population',
       'retail', 'bottles', 'profit_per_sale'],
      dtype='object')


ValueError: Length mismatch: Expected axis has 9 elements, new values have 8 elements

In [ ]:
# Note 
stores.head()

In [ ]:
# FIPS became a float. That decimal is annoying. Let's make it an integer again.
stores.fips=stores.fips.astype(int)

In [ ]:
# How many counties are there?
stores.fips.nunique()

In [ ]:
# How many stores are there in each county?
x=stores.fips.value_counts()
print(len(x))
x.head()

## County-level dataset

In [ ]:
# What's the metro/nonmetro status of each county?
y=stores.groupby('fips').metro.mean()
print(len(y))

In [ ]:
# Is there a relationship between nonmetro status and number of stores in a county?
plt.scatter(x,y)
plt.ylabel('metro')
plt.xlabel('stores')
plt.title('rural counties have more liquor stores')
plt.show

In [ ]:
# Polk county has over 200 liquor stores.
transactions[transactions.fips==19153].head()